In [6]:
import os
from configparser import ConfigParser
from pyrfc import Connection,ABAPRuntimeError

# 1. Locad configuration

In [2]:
configur = ConfigParser()

In [3]:
configur.read('config.ini')

['config.ini']

In [7]:
ASHOST=configur.get('SAP','host')
CLIENT=configur.get('SAP','client')
SYSNR=configur.get('SAP','instance')
USER=configur.get('SAP','username')
PASSWD=configur.get('SAP','password')

In [8]:
conn = Connection(
    ashost=ASHOST, 
    sysnr=SYSNR, 
    client=CLIENT, 
    user=USER, 
    passwd=PASSWD,
    lang="EN"
    )

# Read PO Invoices

In [12]:
def conver_results_to_object(result):
    results = []
    for row in result['DATA']:
        d = row['WA'].split('|')
        fields = result['FIELDS']
        o = {}
        for i,v in enumerate(d):
            o[fields[i]['FIELDNAME']] = v
        results.append(o)
    return results

In [13]:
def get_po_invoices(conn,po):
    po_results = []
    kwargs = dict(
                QUERY_TABLE = 'EKBE',
                DELIMITER = '|',
                NO_DATA = ' ',
                FIELDS = [ 
                    {'FIELDNAME':'EBELN'} ,
                    {'FIELDNAME':'EBELP'} ,
                    {'FIELDNAME':'ZEKKN'} ,
                    {'FIELDNAME':'VGABE'} ,
                    {'FIELDNAME':'GJAHR'} ,
                    {'FIELDNAME':'BELNR'} ,
                    {'FIELDNAME':'BUZEI'} ,
                        ],
                ROWSKIPS = 0 ,
                ROWCOUNT = 0,
                OPTIONS = [
                    "MANDT = '001'", 
                    "EBELN = '"+ po +"'",
                    "VGABE = '2'"
                    ]
            )
    try:
        result = conn.call('RFC_READ_TABLE',**kwargs)
        po_results = conver_results_to_object(result)
    except ABAPRuntimeError as error:
        print(error.message)
    
    return po_results